BY : HAQ NAWAZ MALIK ,

# this is a Notebook Demo , showing how to load any tts dataset that is in the (.zip) or  (.tar.gz ) and mainly contains . txt and and .wav files respectively , to  to prepare a  clean proff. dataset for any use ,

in this below demo i will be showing how to prepare a  clean proff. dataset for any use , and pushing it to hugging face hub ,

In [1]:
#downloading any .tar or zip file
!wget https://openslr.magicdatatech.com/resources/122/kashmiri.tar.gz


--2024-10-15 06:44:11--  https://openslr.magicdatatech.com/resources/122/kashmiri.tar.gz
Resolving openslr.magicdatatech.com (openslr.magicdatatech.com)... 39.96.249.211
Connecting to openslr.magicdatatech.com (openslr.magicdatatech.com)|39.96.249.211|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://openslr.trmal.net/resources/122/kashmiri.tar.gz [following]
--2024-10-15 06:44:12--  https://openslr.trmal.net/resources/122/kashmiri.tar.gz
Resolving openslr.trmal.net (openslr.trmal.net)... 136.243.171.4, 2a01:4f8:171:2012::2
Connecting to openslr.trmal.net (openslr.trmal.net)|136.243.171.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394556614 (376M) [application/x-gzip]
Saving to: ‘kashmiri.tar.gz’

kashmiri.tar.gz     100%[===================>] 376.28M  32.7MB/s    in 12s     

2024-10-15 06:44:25 (31.0 MB/s) - ‘kashmiri.tar.gz’ saved [394556614/394556614]



In [6]:
! pip install datasets
! pip install huggingface_hub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00


In [26]:
import os
import tarfile
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, Audio
import json
import soundfile as sf

def extract_tarfile(tar_path, extract_path):
    with tarfile.open(tar_path, 'r:gz') as tar:
        tar.extractall(path=extract_path)
    print(f"Extracted {tar_path} to {extract_path}")

def create_clean_dataset(source_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)

    data = []
    for filename in os.listdir(source_dir):
        if filename.endswith('.txt'):
            base_name = os.path.splitext(filename)[0]
            txt_path = os.path.join(source_dir, filename)
            wav_path = os.path.join(source_dir, f"{base_name}.wav")

            if os.path.exists(wav_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    text = f.read().strip()

                new_txt_path = os.path.join(dest_dir, filename)
                new_wav_path = os.path.join(dest_dir, f"{base_name}.wav")

                # Copy files
                shutil.copy2(txt_path, new_txt_path)
                shutil.copy2(wav_path, new_wav_path)

                # Verify audio file
                try:
                    audio_info = sf.info(new_wav_path)
                    duration = audio_info.duration
                    sample_rate = audio_info.samplerate
                except Exception as e:
                    print(f"Error processing {new_wav_path}: {str(e)}")
                    continue

                data.append({
                    'file_name': base_name,
                    'text': text,
                    'audio': new_wav_path,
                    'duration': duration,
                    'sample_rate': sample_rate
                })

    df = pd.DataFrame(data)
    print(f"Total files processed: {len(df)}")
    print(f"Total audio duration: {df['duration'].sum():.2f} seconds")
    return df

def split_dataset(df, test_size=0.2, random_state=42):
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)
    return train_df, test_df

def create_metadata(df, output_file):
    metadata = {
        'dataset_name': 'Kashmiri TTS Dataset',
        'language': 'Kashmiri',
        'num_samples': len(df),
        'total_audio_duration': f"{df['duration'].sum():.2f} seconds",
        'file_format': 'WAV',
        'text_format': 'TXT',
        'license': 'CC-BY-4.0',  # Adjust as needed
        'citations': [],  # Add any relevant citations
        'description': 'A Kashmiri text-to-speech dataset containing paired text and audio files.'
    }

    with open(output_file, 'w') as f:
        json.dump(metadata, f, indent=2)

def create_hf_dataset(train_df, test_df, output_dir):
    def process_example(example):
        return {
            'audio': example['audio'],
            'text': example['text'],
            'duration': example['duration'],
            'sample_rate': example['sample_rate']
        }

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    train_dataset = train_dataset.map(process_example)
    test_dataset = test_dataset.map(process_example)

    # Set the audio column format
    train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
    test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

    dataset_dict = DatasetDict({
        'train': train_dataset,
        'test': test_dataset
    })

    dataset_dict.save_to_disk(output_dir)
    print(f"Saved dataset to {output_dir}")

def main():
    source_tar = '/content/kashmiri.tar.gz'
    extract_dir = '/content/kashmiri_tts_dataset'
    clean_dir = '/content/clean_kashmiri_tts_dataset'
    output_dir = '/content/kashmiri_tts_hf_dataset'

    extract_tarfile(source_tar, extract_dir)
    df = create_clean_dataset(extract_dir, clean_dir)
    train_df, test_df = split_dataset(df)
    create_metadata(df, os.path.join(clean_dir, 'metadata.json'))
    create_hf_dataset(train_df, test_df, output_dir)

    print("Processing complete!")

if __name__ == "__main__":
    main()

Extracted /content/kashmiri.tar.gz to /content/kashmiri_tts_dataset
Total files processed: 33
Total audio duration: 9766.49 seconds


Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/26 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7 [00:00<?, ? examples/s]

Saved dataset to /content/kashmiri_tts_hf_dataset
Processing complete!


In [27]:
from datasets import load_from_disk

dataset = load_from_disk('/content/kashmiri_tts_hf_dataset')    # seeing the dataset / replace ypur path
print(dataset)

# To access an audio file:
audio = dataset['train'][0]['audio']
print(audio)

DatasetDict({
    train: Dataset({
        features: ['file_name', 'text', 'audio', 'duration', 'sample_rate', '__index_level_0__'],
        num_rows: 26
    })
    test: Dataset({
        features: ['file_name', 'text', 'audio', 'duration', 'sample_rate', '__index_level_0__'],
        num_rows: 7
    })
})
{'path': 'Irfan_Rashid_193223_25_01_2022 - Ershad.wav', 'array': array([0.        , 0.        , 0.        , ..., 0.00079346, 0.        ,
       0.00030518]), 'sampling_rate': 16000}


# Pussing to hugging face HUb datasets

In [28]:
from datasets import load_from_disk
from huggingface_hub import HfApi, login
import os

def upload_to_hub(dataset_path, repo_name, token):
    # Login to Hugging Face
    login(token)

    # Load the dataset
    dataset = load_from_disk(dataset_path)

    # Push to hub
    dataset.push_to_hub(repo_name, token=token)

    print(f"Dataset uploaded successfully to {repo_name}")

def main():
    # Set your Hugging Face token
    hf_token = "your_hugging_face_token_here"  # Replace with your actual token

    # Set the path to your processed dataset
    dataset_path = "/content/kashmiri_tts_hf_dataset"

    # Set the name for your dataset repository on Hugging Face
    repo_name = "Omarrran/kashmiri_tts_hf_dataset"

    # Upload the dataset
    upload_to_hub(dataset_path, repo_name, hf_token)

if __name__ == "__main__":
    main()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Dataset uploaded successfully to Omarrran/kashmiri_tts_hf_dataset


if  you want see some statistics about the audio files

In [29]:
from datasets import load_dataset
import pandas as pd

def load_and_verify_dataset(dataset_name):
    # Load the dataset from Hugging Face Hub
    dataset = load_dataset(dataset_name)

    print(f"Dataset '{dataset_name}' loaded successfully.")
    print("\nDataset structure:")
    print(dataset)

    # Print information about each split
    for split in dataset.keys():
        print(f"\nInformation for '{split}' split:")
        print(f"Number of samples: {len(dataset[split])}")
        print("Features:")
        for feature, feature_info in dataset[split].features.items():
            print(f"  - {feature}: {feature_info}")

    # Display a few examples
    print("\nFirst few examples:")
    df = pd.DataFrame(dataset['train'][:5])
    print(df)

    # Print some statistics about the audio files
    durations = dataset['train']['duration']
    print("\nAudio statistics:")
    print(f"Total duration: {sum(durations):.2f} seconds")
    print(f"Average duration: {sum(durations) / len(durations):.2f} seconds")
    print(f"Min duration: {min(durations):.2f} seconds")
    print(f"Max duration: {max(durations):.2f} seconds")

# Main execution
dataset_name = "Omarrran/kashmiri_tts_hf_dataset"
load_and_verify_dataset(dataset_name)

README.md:   0%|          | 0.00/658 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/81.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset 'Omarrran/kashmiri_tts_hf_dataset' loaded successfully.

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['file_name', 'text', 'audio', 'duration', 'sample_rate', '__index_level_0__'],
        num_rows: 26
    })
    test: Dataset({
        features: ['file_name', 'text', 'audio', 'duration', 'sample_rate', '__index_level_0__'],
        num_rows: 7
    })
})

Information for 'train' split:
Number of samples: 26
Features:
  - file_name: Value(dtype='string', id=None)
  - text: Value(dtype='string', id=None)
  - audio: Audio(sampling_rate=16000, mono=True, decode=True, id=None)
  - duration: Value(dtype='float64', id=None)
  - sample_rate: Value(dtype='int64', id=None)
  - __index_level_0__: Value(dtype='int64', id=None)

Information for 'test' split:
Number of samples: 7
Features:
  - file_name: Value(dtype='string', id=None)
  - text: Value(dtype='string', id=None)
  - audio: Audio(sampling_rate=16000, mono=True, decode=True, id=None)
  - duration: Value

creating Readme.md for your dataset in colab

In [30]:
from datasets import load_dataset
import pandas as pd
from collections import Counter
import numpy as np

def generate_readme(dataset_name):
    # Load the dataset
    dataset = load_dataset(dataset_name)

    # Initialize the README content
    readme = f"# {dataset_name}\n\n"

    # Dataset description
    readme += "## Dataset Description\n\n"
    readme += "This dataset contains Kashmiri text-to-speech (TTS) data, including paired text and audio files.\n\n"

    # Dataset statistics
    readme += "## Dataset Statistics\n\n"

    total_samples = sum(len(split) for split in dataset.values())
    readme += f"- Total number of samples: {total_samples}\n"

    for split_name, split_data in dataset.items():
        readme += f"- Number of samples in {split_name} split: {len(split_data)}\n"

    # Audio statistics
    durations = [d for split in dataset.values() for d in split['duration']]
    total_duration = sum(durations)
    avg_duration = np.mean(durations)
    min_duration = min(durations)
    max_duration = max(durations)

    readme += f"\n### Audio Statistics\n\n"
    readme += f"- Total audio duration: {total_duration:.2f} seconds ({total_duration/3600:.2f} hours)\n"
    readme += f"- Average audio duration: {avg_duration:.2f} seconds\n"
    readme += f"- Minimum audio duration: {min_duration:.2f} seconds\n"
    readme += f"- Maximum audio duration: {max_duration:.2f} seconds\n"

    # Text statistics
    all_text = [t for split in dataset.values() for t in split['text']]
    total_words = sum(len(text.split()) for text in all_text)
    avg_words = total_words / len(all_text)
    min_words = min(len(text.split()) for text in all_text)
    max_words = max(len(text.split()) for text in all_text)

    readme += f"\n### Text Statistics\n\n"
    readme += f"- Total number of words: {total_words}\n"
    readme += f"- Average number of words per sample: {avg_words:.2f}\n"
    readme += f"- Minimum number of words in a sample: {min_words}\n"
    readme += f"- Maximum number of words in a sample: {max_words}\n"

    # Dataset structure
    readme += "\n## Dataset Structure\n\n"
    readme += "The dataset contains the following fields:\n\n"
    for feature, feature_info in dataset['train'].features.items():
        readme += f"- `{feature}`: {feature_info}\n"

    # Usage example
    readme += "\n## Usage Example\n\n"
    readme += "Here's a simple example of how to load and use this dataset:\n\n"
    readme += "```python\n"
    readme += "from datasets import load_dataset\n\n"
    readme += f"dataset = load_dataset('{dataset_name}')\n\n"
    readme += "# Access the training split\n"
    readme += "train_data = dataset['train']\n\n"
    readme += "# Get the first sample\n"
    readme += "first_sample = train_data[0]\n"
    readme += "print(f\"Text: {first_sample['text']}\")\n"
    readme += "print(f\"Audio file: {first_sample['audio']}\")\n"
    readme += "```\n"

    # License and citation
    readme += "\n## License\n\n"
    readme += "Please refer to the dataset card for license information.\n\n"

    readme += "## Citation\n\n"
    readme += "If you use this dataset in your research, please cite it as follows:\n\n"
    readme += "```\n"
    readme += "@dataset{kashmiri_tts_dataset,\n"
    readme += f"  author = {{Your Name}},\n"
    readme += f"  title = {{{dataset_name}}},\n"
    readme += "  year = {2023},\n"
    readme += f"  url = {{https://huggingface.co/datasets/{dataset_name}}}\n"
    readme += "}\n"
    readme += "```\n"

    return readme

# Generate README
dataset_name = "Omarrran/kashmiri_tts_hf_dataset"
readme_content = generate_readme(dataset_name)

# Print the README content
print(readme_content)

# Optionally, save to a file
with open('README.md', 'w', encoding='utf-8') as f:
    f.write(readme_content)

# Omarrran/kashmiri_tts_hf_dataset

## Dataset Description

This dataset contains Kashmiri text-to-speech (TTS) data, including paired text and audio files.

## Dataset Statistics

- Total number of samples: 33
- Number of samples in train split: 26
- Number of samples in test split: 7

### Audio Statistics

- Total audio duration: 9766.49 seconds (2.71 hours)
- Average audio duration: 295.95 seconds
- Minimum audio duration: 84.60 seconds
- Maximum audio duration: 606.91 seconds

### Text Statistics

- Total number of words: 25290
- Average number of words per sample: 766.36
- Minimum number of words in a sample: 95
- Maximum number of words in a sample: 2100

## Dataset Structure

The dataset contains the following fields:

- `file_name`: Value(dtype='string', id=None)
- `text`: Value(dtype='string', id=None)
- `audio`: Audio(sampling_rate=16000, mono=True, decode=True, id=None)
- `duration`: Value(dtype='float64', id=None)
- `sample_rate`: Value(dtype='int64', id=None)
- `__index_le

loading the dataset and testing finally

In [31]:
from datasets import load_dataset

dataset = load_dataset('Omarrran/kashmiri_tts_hf_dataset')

# Access the training split
train_data = dataset['train']

# Get the first sample
first_sample = train_data[0]
print(f"Text: {first_sample['text']}")
print(f"Audio file: {first_sample['audio']}")


README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7 [00:00<?, ? examples/s]

Text: 2.234073	4.493108	والیکُم اَسَلام مِہ چُھ ناو اِرفان رشید 
5.940887	9.710106	سر بٔہ چُھس روزان ڈسٹِک بانڈیپوٗرا اکِس گامَس یتھ ناو چھ ہٲجن
10.833383	17.211100	بٔہ چُھس ایز اے ریسرچ سکۅلر ڈِپاٹمنٹ آف اوردُوس منٛز کٲم کران  بٔہ چُھس پی ایچ ڈی کران کشمیر یونیورسٹی منٛز
25.135548	30.764022	آو توہِہ چھو یِہ زٲنِتھ زِ ڈِسٹِک بانڈیپورس منٛز چھ اکھ توٲریکھی گام یتھ ناو چھ ہاجِن
31.149194	40.281487	ہاجِن چھ مخطلِف چیزو کِنِہ مشہور رودموت خاص تور سے ہٲجِن چھ یوس ہسا کشمیری زبان چھ تتھ چھ زیاد فٲیِدٔ دِیتموت ہٲجنن
40.554834	51.985731	 ہٲجنس منٛز چھ وہب پرے ہاجنی پرۅفؠسر مٔہدین ہاجنی محمد آسن آسن  یِم چھ تُم شخصیت ہٲجنس منٛز وژھمژ یِمو کشمیری زبانِہ چھ وِز وِز لول بورموت 
52.358478	57.527231	 خاص تور سے یُس ہسا پرۅفؠسر مٔہدین ہاجنی چھ اوسموت  سُہ چھ اوسموت ایز اے پرۅفؠسر 
58.272724	62.273538	 پرتاب یانے ایس پی کالجِہ وارِیہس کالس تکریبن
63.441478	67.392592	  تھٹی ییرس ایز اے پرۅفؠسر سُہ چھ کُنوُہ شتھ تٔہ پنٛژہَس منٛز اؠپونٹ تتِہ گومُت
67.591390	70.672762	تیمِہ چھ کشمیری زبانِہ خٲترٕ وارِیاہ 

# OPTIONAL   "If you get any error for login , use below code for token access directly"

In [7]:
from huggingface_hub import login
from datasets import Dataset
login()